## Load Packages

In [1]:
# Link to Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Connect to Earth Engine
#import ee
#ee.Authenticate()
#ee.Initialize()

In [3]:
!pip install geopandas
!pip install geojson

import os
from glob import glob
import numpy as np
import pandas as pd
import geopandas as gpd
import geojson


     |████████████████████████████████| 1.0 MB 7.6 MB/s 
     |████████████████████████████████| 6.3 MB 40.5 MB/s 
     |████████████████████████████████| 16.7 MB 46.7 MB/s 


## Tidy up Datasets before Mapping/Plotting

### Group zonalStats dataframes by country

In [ ]:
id_KHM = [541, 670]
id_VTM = [530,531,532,533,534,535,536,537,538,540,541]
id_THA = [655,656,657,665,666,667,668,669,670,671,672,695]
id_MYS = [651,658,659,660,662,663,664,618,619,626,627,628,629,630,631]
id_MMR = [655,693,694,695,696,697,698,699,700,701,702]

In [ ]:
def filterRegion(list_id, country):
  """ Merge GeoJSONs and Subset to one country

  Parameters
  ----------
  list_id: list
    List of parcel ids
  country: string
    Country Name
  
  Returns
  ----------
  Exported GeoJSON of one country
  """
  os.chdir("/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/0.Initial/")
  files = [glob(os.path.join(os.getcwd(), '*'+str(i)+'*')) for i in list_id]
  files_flat = [item for sublist in files for item in sublist]
  gdfs = [gpd.read_file(i) for i in files_flat]
  gdfs_filtered = [i[i['NAME_0']==country] for i in gdfs]
  gdf = pd.concat([i for i in gdfs_filtered])

  # Data type of zonStats columns to "uint8"
  #keys = [i for i in gdf.columns if 'majority' in i]
  #values = [np.uint8]*len(keys)
  #dictCol = dict(zip(keys, values))
  #gdf = gdf.astype(dictCol)

  outPath = '/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/1.by_country/'
  return gdf.to_file(os.path.join(outPath, "zonStats_"+str(country)+".geojson"), driver = 'GeoJSON')

In [ ]:
# Write file
#filterRegion(id_KHM, "Cambodia")
filterRegion(id_VTM, "Vietnam")
filterRegion(id_THA, "Thailand")
filterRegion(id_MYS, "Malaysia")
filterRegion(id_MMR, "Myanmar")

In [ ]:
# Draft
#os.chdir("/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/0.Initial/")
#files = [glob(os.path.join(os.getcwd(), '*'+str(i)+'*')) for i in id_cambodia]
#files_flat = [item for sublist in files for item in sublist]
#gdfs = [gpd.read_file(i) for i in files_flat]
#gdfs_filtered = [i[i['NAME_0']=="Cambodia"] for i in gdfs]
#gdf_KHM = pd.concat([i for i in gdfs_filtered])

#keys = [i for i in gdf_KHM.columns if 'majority' in i]
#values = [np.uint8]*len(keys)
#dictCol = dict(zip(keys, values))
#gdf_KHM = gdf_KHM.astype(dictCol)

### Tidy up using GEE
  * Calculate Pond Area, adding as Attributes
  * Transfer Geometry Type from polygon to point

In [ ]:
os.chdir("/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/1.by_country/")
with open('zonStats_Cambodia.geojson') as f:
  geojson_ini = geojson.load(f)

In [ ]:
# Subset big GeoJSON into processible parts (2500 items/part)
geojson_tiled = [tuple(geojson_ini[i:i+2500]) for i in range(0, len(geojson_ini['features']), 2500)]
len(geojson_tiled[-1])

158

In [ ]:
# Load GeoJSON as feature collection
fc = [ee.FeatureCollection(i) for i in geojson_tiled]
type(fc)

list

In [ ]:
# Pond Area in square meters
fc_area = [i.map(lambda feature: feature.set({'Area': feature.area()})) for i in fc]
type(fc_area)

list

In [ ]:
# Get Centroid of Pond Polygons, thus transfer geometry type from polygon to point
fc_point= [i.map(lambda feature: feature.centroid()) for i in fc_area]
type(fc_point)

list

In [ ]:
# Export Feature Collection as GeoJSON
for i in range(len(fc_point)):
  task_config = {
          'description': 'zonStats_KHM_tidy',
          'fileNamePrefix': 'zonStats_KHM_tidy_'+str(i),
          'folder': '2.area_centroid',
          'fileFormat': 'GeoJSON'}

  task = ee.batch.Export.table.toDrive(fc_point[i], **task_config)
  task.start()

In [ ]:
task.status()

### Tidy up using GEE: Function

In [ ]:
def tidy_up(geojson_path):

  # Load GeoJSON
  with open(geojson_path) as f:
    geojson_ini = geojson.load(f)
  
  # Subset big GeoJSON into processible parts (2500 items/part)
  geojson_tiled = [tuple(geojson_ini[i:i+2500]) for i in range(0, len(geojson_ini['features']), 2500)]

  # Load GeoJSON as feature collection
  fc = [ee.FeatureCollection(i) for i in geojson_tiled]

  # Pond Area in square meters
  fc_area = [i.map(lambda feature: feature.set({'Area': feature.area()})) for i in fc]

  # Get Centroid of Pond Polygons, thus transfer geometry type from polygon to point
  fc_point= [i.map(lambda feature: feature.centroid()) for i in fc_area]

  return fc_point

In [ ]:
#countries = ['Malaysia', 'Thailand', 'Vietnam', 'Myanmar']
countries = ['Myanmar']

def genFun(n): # len(countries)

  os.chdir("/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/1.by_country/")
  files = [glob(os.path.join(os.getcwd(), '*'+i+'*.geojson')) for i in countries]

  i = 0
  while i < n:
    input = files[i][0]
    output = tidy_up(input) # output is a list

    for j in range(len(output)):
      task_config = {
              'description': 'zonStats_tidy',
              'fileNamePrefix': 'zonStats_'+countries[i]+'_tidy_'+str(j),
              'folder': '2.area_centroid',
              'fileFormat': 'GeoJSON'}
      task = ee.batch.Export.table.toDrive(output[j], **task_config)
      yield task.start()
      
    i += 1

gen = genFun(len(countries))

In [ ]:
# Exhaust Generator
from collections import deque

def exhaust(generator):
    deque(generator, maxlen=0)

exhaust(gen)

### Merge Dataframes by Country

In [ ]:
countries = ['Malaysia', 'Thailand', 'Vietnam', 'Myanmar', 'KHM']

def genFun(n):
  inPath = "/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/2.area_centroid/"
  outPath = "/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/3.merged_by_country/"

  i = 0
  while i < n:
    files = glob(os.path.join(inPath, '*'+countries[i]+'*.geojson'))
    dfs = [gpd.read_file(i) for i in files]
    df = pd.concat([i for i in dfs])
    yield df.to_file(os.path.join(outPath, countries[i]+"_merged.geojson"))
    i += 1

gen = genFun(len(countries))

In [ ]:
# Exhaust Generator
from collections import deque

def exhaust(generator):
    deque(generator, maxlen=0)

exhaust(gen)

### De-geometry

In [ ]:
os.chdir("/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/3.merged_by_country/")
files = glob(os.path.join(os.getcwd(), '*merged*'))
files

['/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/3.merged_by_country/Malaysia_merged.geojson',
 '/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/3.merged_by_country/Thailand_merged.geojson',
 '/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/3.merged_by_country/Vietnam_merged.geojson',
 '/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/3.merged_by_country/Myanmar_merged.geojson',
 '/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/3.merged_by_country/Cambodia_merged.geojson']

In [ ]:
gdf = gpd.read_file(files[0])

In [ ]:
df = gdf.drop('geometry', 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  """Entry point for launching an IPython kernel.


In [ ]:
def genFun(n): # n=len(files)
  outpath = "/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/4.deGeom/"
  j = 0
  while j < n:
    gdf = gpd.read_file(files[j])
    df = gdf.drop('geometry', 1)
    country = files[j].split('/')[-1].split('_')[0]
    yield df.to_csv(os.path.join(outpath, country+'_degeom.csv'), index=False)
    j += 1

gen = genFun(len(files))

In [ ]:
# Exhaust Generator
# Reference: https://stackoverflow.com/questions/47456631/simpler-way-to-run-a-generator-function-without-caring-about-items
from collections import deque

def exhaust(generator):
    deque(generator, maxlen=0)

exhaust(gen)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


## Plot: time series of active pond numbers & pond area

In [ ]:
os.chdir("/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/4.deGeom/")
files = glob(os.path.join(os.getcwd(), '*'))
files

['/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/4.deGeom/Malaysia_degeom.csv',
 '/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/4.deGeom/Thailand_degeom.csv',
 '/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/4.deGeom/Vietnam_degeom.csv',
 '/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/4.deGeom/Myanmar_degeom.csv',
 '/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/4.deGeom/Cambodia_degeom.csv']

**Note**: There might be ponds whose time series start in 1987, the others in 1988, so the "1987" column might be `NA` for those whose time series start in 1988.

### Flow

In [ ]:
df = pd.read_csv(files[0])
# Fill NA of first column
df[[df.columns[0]]] = df[[df.columns[0]]].fillna(value=2)
# Convert first column to integer
df = df.astype({df.columns[0]:int})

In [ ]:
tsCol = [i for i in df.columns if 'majority' in i]

In [ ]:
active_ponds = [len(df[df[i]==1]) for i in tsCol]

In [ ]:
active_area_km2 = [round(sum(df[df[i]==1]['Area'])/1e6, 2) for i in tsCol]

In [ ]:
years = [i.split('_')[0] for i in tsCol]
country = [files[0].split('/')[-1].split('_')[0]]*len(years)

In [ ]:
data = {'country': country, 
        'year': years,
        'active_count': active_ponds,
        'active_area_km2': active_area_km2}

df_plot = pd.DataFrame(data)

### Function

In [ ]:
def genFun(n): # n=len(files)
  outpath = "/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/6.plot_count_area/"
  j = 0
  while j < n:
    df = pd.read_csv(files[j])
    # Fill NA of first column
    df[[df.columns[0]]] = df[[df.columns[0]]].fillna(value=2)
    # Convert first column to integer
    df = df.astype({df.columns[0]:int})

    tsCol = [i for i in df.columns if 'majority' in i]
    active_ponds = [len(df[df[i]==1]) for i in tsCol]
    active_area_km2 = [round(sum(df[df[i]==1]['Area'])/1e6, 2) for i in tsCol]
    years = [i.split('_')[0] for i in tsCol]
    country = [files[j].split('/')[-1].split('_')[0]]*len(years)

    data = {'country': country, 
            'year': years,
            'active_count': active_ponds,
            'active_area_km2': active_area_km2}
    df_plot = pd.DataFrame(data)

    yield df_plot.to_csv(os.path.join(outpath, country[0]+'_plot_count_area.csv'), index=False)
    j += 1

gen = genFun(len(files))

In [ ]:
# Exhaust Generator
# Reference: https://stackoverflow.com/questions/47456631/simpler-way-to-run-a-generator-function-without-caring-about-items
from collections import deque

def exhaust(generator):
    deque(generator, maxlen=0)

exhaust(gen)

### Merge

In [ ]:
os.chdir("/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/6.plot_count_area/")
files = glob(os.path.join(os.getcwd(), '*'))
files

['/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/6.plot_count_area/Malaysia_plot_count_area.csv',
 '/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/6.plot_count_area/Thailand_plot_count_area.csv',
 '/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/6.plot_count_area/Vietnam_plot_count_area.csv',
 '/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/6.plot_count_area/Myanmar_plot_count_area.csv',
 '/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/6.plot_count_area/Cambodia_plot_count_area.csv']

In [ ]:
dfs = [pd.read_csv(i) for i in files]
df = pd.concat([i for i in dfs])

In [ ]:
#df.to_csv(os.path.join(os.getcwd(), 'plot_count_area_all.csv'), index=False)

## Plot: Statistics of NA in ponds
*(Do it again using de-geom csv file after filling NA with 2 for first time stamp)*

In [ ]:
os.chdir("/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/4.geGeom/")
files = glob(os.path.join(os.getcwd(), '*merged*'))
files

['/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/3.merged_by_country/Malaysia_merged.geojson',
 '/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/3.merged_by_country/Thailand_merged.geojson',
 '/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/3.merged_by_country/Vietnam_merged.geojson',
 '/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/3.merged_by_country/Myanmar_merged.geojson',
 '/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/3.merged_by_country/Cambodia_merged.geojson']

### Flow

In [ ]:
df = gpd.read_file(files[0])

In [ ]:
files[-1].split('/')[-1].split('_')[0]

'Cambodia'

In [ ]:
# Count of Rows
len(df.index)

17658

In [ ]:
col_stats = [i for i in df.columns if 'majority' in i]
years = [i.split('_')[0] for i in col_stats]
na_rate = [len(df[df[i]==2].index)/len(df.index)*100 for i in col_stats]
country = ['Cambodia']*len(years)

In [ ]:
data = {'country': country, 
        'year': years,
        'na_rate': na_rate}

df_NA = pd.DataFrame(data)

### Function

In [ ]:
def genFun(n): # n=len(files)
  j = 0
  #lst = []
  while j < n:
    df = gpd.read_file(files[j])
    col_stats = [i for i in df.columns if 'majority' in i]
    years = [i.split('_')[0] for i in col_stats]
    na_rate = [len(df[df[i]==2].index)/len(df.index)*100 for i in col_stats]
    country = [files[j].split('/')[-1].split('_')[0]]*len(years)
    data = {'country': country, 
            'year': years,
            'na_rate': na_rate}
    df_NA = pd.DataFrame(data)
    yield df_NA.to_csv(os.path.join("/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/5.NA_Stats/", country[0]+'_naStats.csv'), index=False)
    j += 1

gen = genFun(len(files))

In [ ]:
# Exhaust Generator
# Reference: https://stackoverflow.com/questions/47456631/simpler-way-to-run-a-generator-function-without-caring-about-items
from collections import deque

def exhaust(generator):
    deque(generator, maxlen=0)

exhaust(gen)

### Explore

In [ ]:
os.chdir("/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/4.NA_Stats/")
files = glob(os.path.join(os.getcwd(), '*'))
files

['/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/4.NA_Stats/Malaysia_naStats.csv',
 '/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/4.NA_Stats/Thailand_naStats.csv',
 '/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/4.NA_Stats/Vietnam_naStats.csv',
 '/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/4.NA_Stats/Myanmar_naStats.csv',
 '/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/4.NA_Stats/Cambodia_naStats.csv']

In [ ]:
dfs = [pd.read_csv(i) for i in files]
df = pd.concat([i for i in dfs])
df = df.drop('Unnamed: 0', 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
#df.to_csv("/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/4.NA_Stats/naStats_merged.csv", index=False)

In [ ]:
df[df['country']=='Cambodia']

,country,year,na_rate
0,Cambodia,1987,37.365500
1,Cambodia,1988,0.016989
2,Cambodia,1989,0.000000
3,Cambodia,1990,0.000000
4,Cambodia,1991,0.000000
5,Cambodia,1992,0.039642
6,Cambodia,1993,0.000000
7,Cambodia,1994,0.000000
8,Cambodia,1995,0.000000
9,Cambodia,1996,0.000000


## Map: aquaculture pond development by admin region

### Flow

In [12]:
os.chdir("/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/7.map_development/")
countries = ['Cambodia', 'Vietnam', 'Thailand', 'Myanmar', 'Malaysia']

files = [glob(os.path.join(os.getcwd(), '*', '*'+i+'*')) for i in countries]

In [45]:
dfA = pd.read_csv(files[0][0]) # dfA is df of pond zonStats
dfB = gpd.read_file(files[0][1]) # dfB is gdf of admin region

In [21]:
dfB_filtered = dfB[dfB['GID_2'].isin(dfA['GID_2'])]

In [65]:
dfA_1987active = dfA[dfA['1987'+'_majority']==1]
df_area_1987 = dfA_1987active.groupby(['GID_2'], as_index=False)['Area'].sum()
df_area_1987['Area'] = round(df_area_1987['Area']/10e4, 2)
df_area_1987 = df_area_1987.rename(columns={'Area':'Area_ha_1987'})
#df_area_1987

In [58]:
dfB_joined = pd.merge(dfB_filtered, df_area_1987, on='GID_2', how='left')
#dfB_joined

In [24]:
colnames = [col for col in dfA.columns if 'majority' in col]
years = [i.split('_')[0] for i in colnames]
GIDs = dfB_filtered['GID_2'].tolist()

In [68]:
dfB_joined = dfB_filtered
for y in years:
  dfA_1Yactive = dfA[dfA[y+'_majority']==1]

  df_area_1Y = dfA_1Yactive.groupby(['GID_2'], as_index=False)['Area'].sum()
  df_area_1Y['Area'] = round(df_area_1Y['Area']/10e4, 2)
  df_area_1Y = df_area_1Y.rename(columns={'Area':'Area_ha_'+y})

  dfB_joined = pd.merge(dfB_joined, df_area_1Y, on='GID_2', how='left')

### Function

In [77]:
os.chdir("/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/7.map_development/")
countries = ['Cambodia', 'Vietnam', 'Thailand', 'Myanmar', 'Malaysia']

files = [glob(os.path.join(os.getcwd(), '*', '*'+i+'*')) for i in countries]

In [78]:
def genFun(n): # n=len(files)
  j = 0
  while j < n:
    dfA = pd.read_csv(files[j][0]) # dfA is df of pond zonStats
    dfB = gpd.read_file(files[j][1]) # dfB is gdf of admin region

    dfB_filtered = dfB[dfB['GID_2'].isin(dfA['GID_2'])] # Keep only admin regions containing ponds

    colnames = [col for col in dfA.columns if 'majority' in col]
    years = [i.split('_')[0] for i in colnames]
    GIDs = dfB_filtered['GID_2'].tolist()

    dfB_joined = dfB_filtered
    for y in years:
      dfA_1Yactive = dfA[dfA[y+'_majority']==1] # For year Y, select ponds that are active 

      df_area_1Y = dfA_1Yactive.groupby(['GID_2'], as_index=False)['Area'].sum() # Group active ponds by GID and calculate sum of pond area for each GID
      df_area_1Y['Area'] = round(df_area_1Y['Area']/10e4, 2) # Transfer pond area from m2 to ha
      df_area_1Y = df_area_1Y.rename(columns={'Area':'Area_ha_'+y}) # Rename column

      dfB_joined = pd.merge(dfB_joined, df_area_1Y, on='GID_2', how='left') # Join "pond area by GID" of year Y into admin df

    yield dfB_joined.to_file(os.path.join(os.getcwd(), 'dfC_admin_zonStats_aggregated', countries[j]+'_zonStats_byAdmin.geojson'), driver='GeoJSON')
    j += 1

gen = genFun(len(files))

In [79]:
# Exhaust Generator
# Reference: https://stackoverflow.com/questions/47456631/simpler-way-to-run-a-generator-function-without-caring-about-items
from collections import deque

def exhaust(generator):
    deque(generator, maxlen=0)

exhaust(gen)

## Hexagon Map under test

In [ ]:
os.chdir("/content/drive/MyDrive/THESIS_AQUAPONDS/gdf_zonstats/3.merged_by_country/")
hexgrid_ini = gpd.read_file(os.path.join(os.getcwd(), 'hexGrid_KHM.geojson'))
ponds = gpd.read_file(os.path.join(os.getcwd(), 'Cambodia_merged.geojson'))

In [ ]:
hexgrid_ini['hexagonID'] = range(len(hexgrid_ini))

* https://gis.stackexchange.com/questions/358735/how-to-obtain-mean-maximum-and-mininum-of-all-points-located-within-polygons-u

In [ ]:
ponds_sj = gpd.sjoin(ponds, hexgrid_ini, how='left', predicate='within')

In [ ]:
pondsOn_1987 = ponds_sj[ponds_sj['1987_majority']==1]

In [ ]:
col_to_aggregate = [i for i in ponds_sj.columns if 'majority' in i]
new_colnames = ['Area_'+i.split('_')[0] for i in col_to_aggregate]
pondsOn = [ponds_sj[ponds_sj[i]==1] for i in col_to_aggregate]

In [ ]:
d = {'a':1, 'b':2}
for key,val in d.items():
  exec(key + '=val')

In [ ]:
ponds_stats = [i.groupby('hexagonID')['Area'].agg(['sum']) for i in pondsOn]
#ponds_stats = [i.rename({'sum':'Area'}, axis=1, inplace=True) for i,j in zip(ponds_stats, new_colnames)]
len(ponds_stats)

33

In [ ]:
ponds_stats[3].info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 662 entries, 32.0 to 3204.0
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   sum     662 non-null    float64
dtypes: float64(1)
memory usage: 10.3 KB


In [ ]:
ponds_stats[32]

In [ ]:

ponds_stats = pondsOn_1987.groupby('hexagonID').agg(Area_1987 = ('Area','sum'))
ponds_stats

,Area_1987
hexagonID,
32.0,7.204881e+05
33.0,1.019850e+06
34.0,8.708514e+04
35.0,5.076738e+03
78.0,4.843661e+04
...,...
2996.0,2.247707e+03
3050.0,2.838944e+03
3051.0,4.004482e+03
